In [ ]:
from dotenv import load_dotenv
import os

In [122]:
# Load variables from .env into environment
load_dotenv()

# Access them with os.environ
api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")
debug_mode = os.getenv("DEBUG") == "True"

paper = True
data_api_url = None

In [ ]:
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np

In [117]:
stock_historical_data_client = StockHistoricalDataClient(
    api_key, secret_key, url_override=data_api_url
)

symbol = "GOOGL"

# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/stockbars-1
now = datetime.now(ZoneInfo("America/Chicago"))
req = StockBarsRequest(
    symbol_or_symbols=[symbol],
    timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),  # specify timeframe
    start=now
    - timedelta(
        weeks=52 * 4
    ),  # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    # limit=1000,  # specify limit
)

df_bars = stock_historical_data_client.get_stock_bars(req).df

# Calculate 5-period and 20-period Simple Moving Average (SMA)
df_bars["SMA_5"] = df_bars["close"].rolling(window=5).mean()
df_bars["SMA_20"] = df_bars["close"].rolling(window=20).mean()

# Exclude days where SMA values are NaN
df_bars = df_bars.dropna(subset=["SMA_5", "SMA_20"])

print(f"Number of rows after dropping NaNs: {len(df_bars)}")

Number of rows after dropping NaNs: 981


In [111]:
df_bars

open       high       low    close  \
symbol timestamp                                                           
GOOGL  2018-10-10 04:00:00+00:00  1136.400  1137.0200  1091.510  1092.16   
       2018-10-11 04:00:00+00:00  1079.040  1116.0550  1075.890  1090.74   
       2018-10-12 04:00:00+00:00  1119.640  1125.8299  1095.025  1120.54   
       2018-10-15 04:00:00+00:00  1118.000  1121.7950  1098.220  1102.44   
       2018-10-16 04:00:00+00:00  1113.480  1136.3700  1111.750  1133.08   
...                                    ...        ...       ...      ...   
       2025-09-04 04:00:00+00:00   229.650   232.3700   226.110   232.30   
       2025-09-05 04:00:00+00:00   232.200   235.7600   231.900   235.00   
       2025-09-08 04:00:00+00:00   235.470   238.1300   233.670   234.04   
       2025-09-09 04:00:00+00:00   234.165   240.4700   233.229   239.63   
       2025-09-10 04:00:00+00:00   238.900   241.6600   237.850   239.17   

                                      volume  trade_count         vwap  \
symbol timestamp                                                         
GOOGL  2018-10-10 04:00:00+00:00   3214009.0      79265.0  1109.379482   
       2018-10-11 04:00:00+00:00   3616807.0      87414.0  1094.577820   
       2018-10-12 04:00:00+00:00   2675095.0      70392.0  1114.923713   
       2018-10-15 04:00:00+00:00   1656823.0      41372.0  1107.779159   
       2018-10-16 04:00:00+00:00   2075030.0      59575.0  1127.458792   
...                                      ...          ...          ...   
       2025-09-04 04:00:00+00:00  51684167.0     648878.0   229.646269   
       2025-09-05 04:00:00+00:00  46324706.0     689004.0   234.083731   
       2025-09-08 04:00:00+00:00  32474743.0     521219.0   235.234574   
       2025-09-09 04:00:00+00:00  38060959.0     527732.0   237.855699   
       2025-09-10 04:00:00+00:00  35141191.0     514898.0   239.713609   

                                     SMA_5     SMA_20  
symbol timestamp                                       
GOOGL  2018-10-10 04:00:00+00:00  1147.630  1178.6410  
       2018-10-11 04:00:00+00:00  1130.364  1174.0710  
       2018-10-12 04:00:00+00:00  1120.906  1171.1990  
       2018-10-15 04:00:00+00:00  1110.210  1168.3295  
       2018-10-16 04:00:00+00:00  1107.792  1166.6280  
...                                    ...        ...  
       2025-09-04 04:00:00+00:00   219.772   207.1640  
       2025-09-05 04:00:00+00:00   224.444   209.0880  
       2025-09-08 04:00:00+00:00   228.670   210.7190  
       2025-09-09 04:00:00+00:00   234.326   212.6505  
       2025-09-10 04:00:00+00:00   236.028   214.4420  

[1738 rows x 9 columns]

In [112]:
# Ensure sorted
df = df_bars.sort_index(level=["symbol", "timestamp"])

# 1) Today: Close > Open
is_up = df["close"] > df["open"]

# 2) Previous day also Close > Open
prev_up = is_up.groupby(level="symbol").shift(1)

# 3) Open below SMA_5 AND Close above SMA_5
cross_above_sma5 = (df["open"] < df["SMA_5"]) & (df["close"] > df["SMA_5"])

# 4) SMA_5 below SMA_20
sma_trend_ok = df["SMA_5"] < df["SMA_20"]

# 5) Previous day Close < SMA_5
prev_close_below_sma5 = (df["close"] < df["SMA_5"]).groupby(level="symbol").shift(1)

# 6) Current AND previous day Close < SMA_20
close_below_sma20 = df["close"] < df["SMA_20"]
prev_close_below_sma20 = close_below_sma20.groupby(level="symbol").shift(1)
both_close_below_sma20 = close_below_sma20 & prev_close_below_sma20

# --- Combine all conditions ---
mask = (
    is_up
    & prev_up
    & cross_above_sma5
    & sma_trend_ok
    & prev_close_below_sma5
    & both_close_below_sma20
)

# --- Filter results ---
filtered = df[mask].reset_index()  # brings symbol, timestamp into columns

# Per-symbol lists of dates
per_symbol_dates = filtered.groupby("symbol")["timestamp"].apply(list)

# Flat list of dates (across all symbols)
dates_list = filtered["timestamp"].tolist()

In [113]:
dates_list

[Timestamp('2018-10-31 04:00:00+0000', tz='UTC'),
 Timestamp('2018-12-26 05:00:00+0000', tz='UTC'),
 Timestamp('2019-05-06 04:00:00+0000', tz='UTC'),
 Timestamp('2019-10-04 04:00:00+0000', tz='UTC'),
 Timestamp('2020-03-02 05:00:00+0000', tz='UTC'),
 Timestamp('2020-06-30 04:00:00+0000', tz='UTC'),
 Timestamp('2020-08-06 04:00:00+0000', tz='UTC'),
 Timestamp('2020-09-25 04:00:00+0000', tz='UTC'),
 Timestamp('2021-03-23 04:00:00+0000', tz='UTC'),
 Timestamp('2022-01-11 05:00:00+0000', tz='UTC'),
 Timestamp('2022-04-19 04:00:00+0000', tz='UTC'),
 Timestamp('2022-08-25 04:00:00+0000', tz='UTC'),
 Timestamp('2022-10-13 04:00:00+0000', tz='UTC'),
 Timestamp('2023-02-15 05:00:00+0000', tz='UTC'),
 Timestamp('2023-03-01 05:00:00+0000', tz='UTC'),
 Timestamp('2023-08-22 04:00:00+0000', tz='UTC'),
 Timestamp('2024-08-16 04:00:00+0000', tz='UTC'),
 Timestamp('2024-11-26 05:00:00+0000', tz='UTC'),
 Timestamp('2025-02-13 05:00:00+0000', tz='UTC'),
 Timestamp('2025-03-21 04:00:00+0000', tz='UTC')]